# <font color='green'>Metal-Organic Frame Materials Prediction</font>
___________________________
## Redes Neurais
___________________________
#### Alunos: Cauê Gomes Correia Santos, Izaque Junior Oliveira Silva e Karla Rovedo Pascoalini
#### Professor: Daniel Roberto Cassar
_____

**Resumo do Problema**

**De onde vieram nossos dados?**

**Features**

**Targets**

**O que são MOFs**

**O que utilizamos como base para o código**

____
**Montar o script**

Divisão do notebook:

[1] Adquirindo dados

[3] Tratamento de Dados

[2] Analisando dados

[4] Diminuindo Dimensionalidade

[5] Separando Target e Features

[6] Implementando a Rede Neural

[7] Curva de Aprendizado

[8] Otimização de Hiperparâmetros 

[9] Cálculo RMSE

[10] Conclusão

[11] Agradecimentos

[12] Referências bibliográficas

In [1099]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem as al
from rdkit.Chem import Draw, AllChem, Descriptors
from cod_izaque import converte
import numpy as np
from sklearn.model_selection import train_test_split
import lightning as L
import matplotlib.pyplot as plt
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MaxAbsScaler
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset

## 1-  Adquirindo Dados

___

Os nossos dados adveêm do site `Kaggle` e fazem parte da tarefa: `Metal-Organic Frame Materials Prediction`

In [1100]:
dados_treino_finger = pd.read_csv(r"dados_treino\finger_train.csv", sep=',', header=0)
dados_treino_RAC = pd.read_csv(r"dados_treino\RAC_train.csv", sep=',', header=0)
dados_teste_RAC = pd.read_csv(r"dados_teste\RAC_test.csv", sep=',', header=0)
dados_teste_finger = pd.read_csv(r"dados_teste\finger_test.csv", sep=',', header=0)
dados_conf_ele_correct = pd.read_csv("electronic_configuration.csv", sep=',', header=0)

## 2 - Análise de Dados
___

### Identificando Dados presentes em ambos DFs

In [1101]:
def enc_intersection(dados1, dados2):
    a = set(list(dados1))
    b = set(list(dados2))
    
    c = a.intersection(b)
    
    return c

In [1102]:
intercecao_treino = enc_intersection(dados_treino_finger['mof'].values, dados_treino_RAC['mof'].values)
intercecao_teste = enc_intersection(dados_teste_finger['mof'].values, dados_teste_RAC['mof'].values)

In [1103]:
#aplicando filtro para 
dados_fingertrain_filtrado = dados_treino_finger.loc[dados_treino_finger['mof'].isin(intercecao_treino)] #referncia: https://pt.stackoverflow.com/questions/309037/filtrar-linhas-no-pandas-por-uma-lista
dados_ractrain_filtrado = dados_treino_RAC.loc[dados_treino_RAC['mof'].isin(intercecao_treino)]
dados_fingertest_filtrado = dados_teste_finger.loc[dados_teste_finger['mof'].isin(intercecao_teste)] #referncia: https://pt.stackoverflow.com/questions/309037/filtrar-linhas-no-pandas-por-uma-lista
dados_ractest_filtrado = dados_teste_RAC.loc[dados_teste_RAC['mof'].isin(intercecao_teste)]

### Verificando o tipo de dado no DataFrame

In [1104]:
dados_fingertrain_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361 entries, 0 to 513
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mof              361 non-null    object 
 1   1s               361 non-null    int64  
 2   2s               361 non-null    int64  
 3   3s               361 non-null    int64  
 4   4s               361 non-null    int64  
 5   5s               361 non-null    int64  
 6   6s               361 non-null    int64  
 7   7s               361 non-null    int64  
 8   2p               361 non-null    int64  
 9   3p               361 non-null    int64  
 10  4p               361 non-null    int64  
 11  5p               361 non-null    int64  
 12  6p               361 non-null    int64  
 13  3d               361 non-null    int64  
 14  4d               361 non-null    int64  
 15  5d               361 non-null    int64  
 16  6d               361 non-null    int64  
 17  4f              

In [1105]:
dados_ractrain_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361 entries, 0 to 534
Columns: 354 entries, mof to additive_category
dtypes: float64(260), int64(93), object(1)
memory usage: 1001.2+ KB


In [1106]:
dados_fingertest_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 0 to 133
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   mof              134 non-null    int64 
 1   1s               134 non-null    int64 
 2   2s               134 non-null    int64 
 3   3s               134 non-null    int64 
 4   4s               134 non-null    int64 
 5   5s               134 non-null    int64 
 6   6s               134 non-null    int64 
 7   7s               134 non-null    int64 
 8   2p               134 non-null    int64 
 9   3p               134 non-null    int64 
 10  4p               134 non-null    int64 
 11  5p               134 non-null    int64 
 12  6p               134 non-null    int64 
 13  3d               134 non-null    int64 
 14  4d               134 non-null    int64 
 15  5d               134 non-null    int64 
 16  6d               134 non-null    int64 
 17  4f               134 non-null    in

In [1107]:
dados_ractest_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 0 to 133
Columns: 342 entries, mof to CH4HPSTP
dtypes: float64(255), int64(87)
memory usage: 359.1 KB


### Encontrando os dados `object` e convertendo

In [1108]:
dados_fingertrain_filtrado.select_dtypes(include='object')

,mof,metal,linker1smi
0,WIRMOB_clean,Zn,O=C(O)c1ccc(COc2c3c(c(OCc4ccc(C(=O)O)cc4)c4c2C...
1,VIXTUT_clean,Zn,c1cc(CCc2ccncc2)ccn1
2,PODKUQ_clean,Cu,O=C(O)c1cc(C(=O)O)cc(-n2cc(-c3ccc(-c4cc(-c5ccc...
3,IKUTOZ_clean,Dy,O=C(O)c1c2ccccc2c(C(=O)O)c2ccccc12
4,LADHEF_clean,Zn,c1nnc[nH]1
...,...,...,...
507,LIKGUJ_clean,Cu,O=C1OC(=O)C2=C3C1=CC1C(=O)OC(=O)C4=C1C3C1=C(C4...
508,ZALMIL_clean,Co,O=C(O)C1CC(C(=O)O)C(C(=O)O)CC1C(=O)O
509,BINROH_clean,Cu,O=C(O)c1ccc2cc(-c3cc(-c4ccc5cc(C(=O)O)ccc5c4)c...
511,TAGTUT_clean,Mg,O=C(O)c1ccc2cc(C(=O)O)ccc2c1


In [1109]:
dados_fingertest_filtrado.select_dtypes(include='object')

,metal,linker1smi
0,Zn,Nc1ncnc2nc[nH]c12
1,Cu,O=C(O)c1cc(C(=O)O)cc(-c2cc(-c3cc(C(=O)O)cc(C(=...
2,Mg,O=C(O)c1ccc2cc(C(=O)O)ccc2c1
3,Gd,O=C(O)c1ccc2[nH]c(-c3ccc(-c4nc5ccc(C(=O)O)cc5[...
4,Cu,O=C(O)c1cncc(C(=O)O)c1
...,...,...
129,Fe,[O-]C(=O)C1=[C]C2=C(S1)C1=[C]C(=[C]C1=C1[C]2[C...
130,Fe,[O-]C(=O)c1cc(C=Cc2cc(cc(c2)C(=O)O)C(=O)[O-])c...
131,Mn,[O-]C(=O)c1ccc(cc1)[Si](c1ccc(cc1)C(=O)[O-])(c...
132,Cd,[O-]C(=O)c1ccc(cc1)[Si](c1ccc(cc1)C(=O)[O-])(c...


In [1110]:
dados_ractrain_filtrado.select_dtypes(include='object')

,mof
0,QUHMOX_clean
1,VOLPET_clean
2,XUHVON_clean
3,KIDDOS_clean
4,FURFOP_clean
...,...
530,YIPDOR_clean
531,EREGOY_clean
532,MAYKOO_clean
533,ADASEF_clean


As colunas que o pandas identifica como do tipo `object` são todas do tipo `string`. Vamos convertê-las.

In [1111]:
lista = ['mof', 'metal', 'linker1smi']

def converte_datatype(df, lista_com_colunas_covrt, type_converte):
    for coluna in lista_com_colunas_covrt:
        df[coluna] = converte(df[coluna], type_converte)

In [1112]:
converte_datatype(dados_fingertrain_filtrado, lista, 'string')
converte_datatype(dados_fingertest_filtrado, lista[1:], 'string')
converte_datatype(dados_ractrain_filtrado, lista[:1], 'string')

C:\Users\izaque23003\AppData\Local\Temp\ipykernel_12472\1425689555.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna] = converte(df[coluna], type_converte)


### Verificando presença de dados não preenchidos

In [1113]:
print(list(dados_fingertrain_filtrado.isnull().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [1114]:
print(list(dados_fingertest_filtrado.isnull().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [1115]:
print(list(dados_ractrain_filtrado.isnull().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [1116]:
print(list(dados_ractest_filtrado.isnull().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## 3 - Tratamento de Dados
___

### Antes da diversão, alguns ajustes...

Para a implementação do `VIF` é necessário dados numéricos e nosso dataset possui uma mescla de dados str e dados int/float. Alguns dados str/object devem ser mantidos e para isso vamos realizar uma conversão simbólico-numérico, este é o caso do `metal`. A coluna `linker1smi` será transformada em um número com o auxílio da biblioteca `rdkit` com o módulo `Chem`.

In [1117]:
lista_rac = list(dados_ractrain_filtrado.columns)
lista_rac.remove('mof')

In [1118]:
lista= list(dados_fingertrain_filtrado.columns)
lista.remove('mof')

In [1119]:
lista_teste = list(dados_fingertest_filtrado.columns)
lista_teste.remove('mof')

In [1120]:
lista_rac_teste = list(dados_ractest_filtrado)
lista_rac_teste.remove('mof')

In [1121]:
treino_finger_cp = dados_fingertrain_filtrado[lista]
treino_rac_cp = dados_ractrain_filtrado[lista_rac]
teste_finger_cp = dados_fingertest_filtrado[lista_teste]
teste_rac_cp = dados_ractest_filtrado[lista_rac_teste]

## <font color='red'>Atenção!</font>
Os códigos abaixo que dizem respeito à exploração dos dados contidos na coluna `linker1smi` foram baseados nos códigos de um usuário do KAGGLE `Marquis03`.

Como era de se imaginar, ele adicionou uma coluna para cada bit do fingerprint. Sabendo que todas as informações que necessitamos foram recolhidas, não vejo a necessidade de aumentar tanto o número de `Features` do nosso dataset.

In [1125]:
def stri_to_mol(df, with_linker1smi=False):
    
    if with_linker1smi==False:
        #print('Estou no if false')
        molecula = df['linker1smi'].apply(Chem.MolFromSmiles)
        df = df.drop('linker1smi', axis=1)
        df = df.drop('metal', axis=1)
        #print('Df antes de novos_parametros', df) #debug
        parametros_novos(df, molecula)
        #print('Df depois de novos_parametros', df)#debug
    if with_linker1smi==True:
        
        molecula = df['linker1smi'].apply(Chem.MolFromSmiles)
        df =df.drop('metal', axis=1)
        fingerprint = lambda x: al.GetMorganFingerprintAsBitVect(x, 3)
        #baseado no código do Marquis03
        fingerprint1 = np.array(molecula.apply(fingerprint).tolist())             
        fingerprint1 = pd.DataFrame(fingerprint1).add_prefix("fp1_")
        fingerprint1.index = df.index
        df =df.drop('linker1smi', axis=1)
        df = pd.concat([df, fingerprint1], axis=1)
        #Fim
        parametros_novos(df, molecula)
    
    return df
    
def parametros_novos(df, molecula):
    df['Peso Molecula'] = molecula.apply(Descriptors.ExactMolWt)
    df['TPSA'] = molecula.apply(Chem.rdMolDescriptors.CalcTPSA) #Total Superficial Polar Area
    df['HBA'] = molecula.apply(Chem.rdMolDescriptors.CalcNumHBA) #Hidrogen bond acceptors
    df['HBD'] = molecula.apply(Chem.rdMolDescriptors.CalcNumHBD) #Hidrogen bond donnors
    df['LogP'] = molecula.apply(Descriptors.MolLogP) #calculus the hidrophobicity
    df['RotnBond'] = molecula.apply(Chem.rdMolDescriptors.CalcNumRotatableBonds) #calculus the number of rotation bonds

In [1126]:
treino_finger_with_linker = treino_finger_cp.copy()
teste_finger_with_linker = teste_finger_cp.copy()

In [1127]:
treino_finger_cp = stri_to_mol(treino_finger_cp)
teste_finger_cp = stri_to_mol(teste_finger_cp)
treino_finger_with_linker = stri_to_mol(treino_finger_with_linker, with_linker1smi =True)
teste_finger_with_linker = stri_to_mol(teste_finger_with_linker, with_linker1smi =True)

Com os valores de `linker1smi` convertidos, temos agora todas as informações sobre cada molécula presente no `dataset`. Do código que nos baseamos, ele adicionava os dados do array do `fingerprint` como colunas separadamente. Vamos tentar partir de outro ponto, nós vamos retirar a coluna `linker1smi` e realizar o VIF para diminuir a multicolinearidade e o número de dimensões.

## 4 - Reduzindo a Dimensionalidade com o VIF
___

In [1128]:
#Créditos: Daniel Roberto Cassar
from sklearn.linear_model import LinearRegression

def selecao_vif(df_atributos, limiar_vif):
    """Realiza a seleção de atributos por VIF.

    Args:
      df_atributos: DataFrame contendo os atributos.
      limiar_vf: valor do limiar do vif. Número positivo. Usualmente é 5 ou 10.

    Returns:
      DataFrame com os atributos selecionados.
    """

    df = df_atributos.copy()

    while True:
        VIFs = []

        for col in df.columns:
            X = df.drop(col, axis=1).values
            y = df[col].values

            r_quadrado = LinearRegression().fit(X, y).score(X, y)

            if r_quadrado != 1:
                VIF = 1 / (1 - r_quadrado)
            else:
                VIF = float("inf")

            VIFs.append(VIF)

        VIF_maximo = max(VIFs)

        if VIF_maximo > limiar_vif:
            indice = VIFs.index(VIF_maximo)
            coluna_remocao = df.columns[indice]
            df = df.drop(coluna_remocao, axis=1)
        else:
            break

    return df

In [1129]:
#from cod_izaque import selecao_fif
#VIF para dados_treino_finger
LIMIAR_VIF = 5

VIF_treino_finger = selecao_vif(treino_finger_cp, LIMIAR_VIF)

In [1130]:
VIF_treino_finger #DATASET DE TREINO 1

,4s,3p,3d,4d,5d,4f,oxidation_state,temperature,time,HBA,HBD,LogP
0,2,6,10,0,0,0,2,110,72.0,4,2,10.21540
1,2,6,10,0,0,0,2,85,72.0,2,0,2.26180
2,1,6,10,0,0,0,2,80,24.0,15,6,8.61990
3,2,6,10,10,0,10,3,95,24.0,2,2,3.38940
4,2,6,10,0,0,0,2,80,72.0,2,1,-0.19530
...,...,...,...,...,...,...,...,...,...,...,...,...
507,1,6,10,0,0,0,2,65,48.0,9,0,-0.35750
508,2,6,7,0,0,0,2,140,72.0,4,4,-0.41660
509,1,6,10,0,0,0,2,90,24.0,3,3,9.24180
511,0,0,0,0,0,0,2,120,120.0,2,2,2.23620


In [1131]:
#from cod_izaque import selecao_fif
#VIF para dados_treino_finger
LIMIAR_VIF = 5

VIF_teste_finger = selecao_vif(teste_finger_cp, LIMIAR_VIF)

In [1132]:
VIF_teste_finger #DATASET DE TESTE 1

,4s,6s,3p,6p,3d,4d,5d,oxidation_state,Peso Molecula,HBD,LogP,RotnBond
0,2,0,6,0,10,0,0,2,135.054495,2,-0.06490,0
1,1,0,6,0,10,0,0,2,570.079826,6,4.87680,9
2,0,0,0,0,0,0,0,2,216.042259,2,2.23620,2
3,2,2,6,0,10,10,1,3,398.101505,4,4.16960,4
4,1,0,6,0,10,0,0,1,167.021858,2,0.47800,2
...,...,...,...,...,...,...,...,...,...,...,...,...
129,2,0,6,6,0,0,0,3,351.915275,0,-2.03445,3
130,2,0,6,0,6,0,0,3,353.031388,1,-1.35430,6
131,0,0,6,0,5,0,0,2,508.063638,0,-3.48200,8
132,2,0,6,0,10,10,0,2,508.063638,0,-3.48200,8


## 5 - Separando `TARGET` e `FEATURES`
___

In [ ]:
TAMANHO_TESTE = 0.1
TAMANHO_VALIDACAO = 0.2
SEMENTE_ALEATORIA = 1234567890

In [ ]:
TARGET_treino = VIF_treino_finger[['temperature', 'time']]

colunas = list(VIF_treino_finger.columns)
colunas.remove('temperature')
colunas.remove('time')

FEATURES_treino = VIF_treino_finger[colunas]

TARGET_teste = VIF_teste_finger[['temperature', 'time']]

colunas = list(VIF_teste_finger.columns)
colunas.remove('temperature')
colunas.remove('time')

FEATURES_teste = VIF_treino_finger[colunas]

# SPLITANDO PARA SEPARAR DF DE VALIDAÇÃO

FEATURES_teste, FEATURES_validacao, TARGET_teste, TARGET_validacao = train_test_split(FEATURES_teste, TARGET_teste, test_size=TAMANHO_VALIDACAO, random_state=SEMENTE_ALEATORIA)

### Normalização e Transformação em tensor

In [ ]:
# NORMALIZANDO DE -1 a 1 PQ QUEREMOS GRANDEZAS QUE VALEM 0 CONTINUEM 0.
x_padrao = MaxAbsScaler()
x_padrao.fit(FEATURES_treino)

y_padrao = MaxAbsScaler()
y_padrao.fit(TARGET_treino)

FEATURES_treino = x_padrao.transform(FEATURES_treino)
TARGET_treino = y_padrao.transform(TARGET_treino)

FEATURES_validacao = x_padrao.transform(FEATURES_validacao)
TARGET_validacao = y_padrao.transform(TARGET_validacao)

FEATURES_teste = x_padrao.transform(FEATURES_teste)
TARGET_teste = y_padrao.transform(TARGET_teste)

In [ ]:
# TORNANDO-O TENSOR
FEATURES_treino = torch.tensor(FEATURES_treino, dtype=torch.float32)
TARGET_treino = torch.tensor(TARGET_treino, dtype=torch.float32)

FEATURES_teste = torch.tensor(FEATURES_teste, dtype=torch.float32)
TARGET_teste = torch.tensor(TARGET_teste, dtype=torch.float32)

FEATURES_validacao = torch.tensor(FEATURES_validacao, dtype=torch.float32)
TARGET_validacao = torch.tensor(TARGET_validacao, dtype=torch.float32)

___
## <font color='red'>Atenção!</font>

Os códigos abaixo foram escritos/inspirados a partir dos códigos dos cadernos jupyter do professor Daniel Roberto Cassar$.^{[ref]}$
___

## 6 - Implementação da Rede Neural
___

### `DataModule` Simplificado

In [ ]:
#Código escrito por Daniel Roberto Cassar
class DataModule(L.LightningDataModule):  
    def __init__(
        self,
        X_treino,
        y_treino,
        X_val,
        y_val,
        X_teste,
        y_teste,
        tamanho_lote = 256,
        num_trabalhadores = 2,
    ):
        super().__init__()

        self.tamanho_lote = tamanho_lote
        self.num_trabalhadores = num_trabalhadores

        self.X_treino = X_treino
        self.y_treino = y_treino
        self.X_val = X_val
        self.y_val = y_val
        self.X_teste = X_teste
        self.y_teste = y_teste

    def train_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_treino, self.y_treino),
            batch_size=self.tamanho_lote,
            num_workers=self.num_trabalhadores,
        )

    def val_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_val, self.y_val),
            batch_size=self.tamanho_lote,
            num_workers=self.num_trabalhadores,
        )

    def test_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_teste, self.y_teste),
            batch_size=self.tamanho_lote,
            num_workers=self.num_trabalhadores,
        )

### Criando a Rede Neural com `Pytorch Lightning`

In [ ]:
#Código escrito por Daniel Roberto Cassar
class MLP(L.LightningModule):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        super().__init__()

        self.camadas = nn.Sequential(
            nn.Linear(num_dados_entrada, neuronios_c1),
            nn.Sigmoid(),
            nn.Linear(neuronios_c1, neuronios_c2),
            nn.Sigmoid(),
            nn.Linear(neuronios_c2, num_targets),
        )

        self.fun_perda = F.mse_loss

        self.perdas_treino = []
        self.perdas_val = []

        self.curva_aprendizado_treino = []
        self.curva_aprendizado_val = []

    def forward(self, x):
        x = self.camadas(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y, y_pred)

        self.log("loss", loss, prog_bar=True)
        self.perdas_treino.append(loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y, y_pred)

        self.log("val_loss", loss, prog_bar=True)
        self.perdas_val.append(loss)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y, y_pred)

        self.log("test_loss", loss)

        return loss

    def on_train_epoch_end(self):
        perda_media = torch.stack(self.perdas_treino).mean()
        self.curva_aprendizado_treino.append(float(perda_media))
        self.perdas_treino.clear()

    def on_validation_epoch_end(self):
        perda_media = torch.stack(self.perdas_val).mean()
        self.curva_aprendizado_val.append(float(perda_media))
        self.perdas_val.clear()

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=1e-3)
        return optimizer

### Treinando a Rede

In [ ]:
NUM_EPOCAS = 2
TAXA_DE_APRENDIZADO = 0.5
NUM_DADOS_DE_ENTRADA = 15
NUM_DADOS_DE_SAIDA = 1
NEURONIO1 = 112
NEURONIO2 = 2

treinador = L.Trainer(max_epochs=NUM_EPOCAS)

dm = DataModule(FEATURES_treino, TARGET_treino, FEATURES_validacao, TARGET_validacao, FEATURES_teste, TARGET_teste) # CRIANDO INSTANCIA DATAMODULE

minha_mlp = MLP(NUM_DADOS_DE_ENTRADA, NEURONIO1, NEURONIO2, NUM_DADOS_DE_SAIDA)
# CRIANDO INSTÂNCIA DA REDE NEURAL

In [ ]:
treinador.fit(minha_mlp, dm) #FITANDO E FORNECENDO (TAMBEM) UMA INSTANCIA DATAMODULE COMO
# ARGUMENTO

## 7 - Curva de Aprendizado
___

In [ ]:
figura, eixo = plt.subplots()

ca_treino = minha_mlp.curva_aprendizado_treino
ca_val = minha_mlp.curva_aprendizado_val

eixo.plot(ca_treino, label="Treino")
eixo.plot(ca_val, label="Validação")

eixo.legend()

eixo.set_xlim(left=0)

eixo.set_title("Curva de aprendizado")
eixo.set_xlabel("Época")
eixo.set_ylabel("Loss")
#N VAI RODAR NO PCZINHO DA ILUM :(

## 9 - Cálculo de RMSE
___

In [ ]:
minha_mlp.eval()

In [ ]:
#NÃO COMPUTA GRADIENTE LOCAL
#COMPUTA: VALOR PREVISTO DO TARGET E DESNORMALIZA OS DADOS

dm.setup("test")

with torch.no_grad():
    X_true = dm.X_teste

    y_true = dm.y_teste
    y_true = dm.y_scaler.inverse_transform(y_true)

    y_pred = minha_mlp(X_true)
    y_pred = dm.y_scaler.inverse_transform(y_pred)

    RMSE = mean_squared_error(y_true, y_pred, squared=False)

    print(RMSE)

# 10 - Conclusão
___

# 11 - Agradecimentos
___

# 12 - Referências Bibliográficas
___